In [91]:
######################################################################################################
# Loading libraries ##################################################################################
######################################################################################################
import argparse, math, random, simanneal, sys
import networkx as nx
import numpy as np
import pandas as pd
from __future__ import print_function
from math import log
from simanneal import Annealer
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import hamming

#####################################################################################################
# Defining constants ################################################################################
######################################################################################################
TAU = 0.15 # Used in PageRank calculation (alpha is 1-tau where [RAB2009 says \tau=0.15])
PAGE_RANK = 'page_rank'
MODULE_ID = 'module_id'

#####################################################################################################
# Auxiliary functions definition ####################################################################
######################################################################################################
## It's generally a good practice not to use numbers in functions' names 
##   (just pedantic CS rules that once were important because of old architectures).
def log_base_two(probability):
    "Returns the log of probability in base 2"
    base_two_logarithm = log(probability,2)
    return base_two_logarithm 

## Using only one 'return' statement is safer than branching them in conditional statements. 
##   It does not make a difference here, but in larger projects it makes code more readable 
##   (and 'friendlier' to collaborators).
def partial_entropy(probability):
    "Half of the entropy function, as used in the InfoMap paper."
    partial_entropy = 0
    if probability != 0:
        partial_entropy = probability * log2(probability)
    return partial_entropy

## Splitting this large function into 3 sub-functions. It's generally a good practice to split large 
##   functions into smaller ones: one function = one specific goal. In this case, having three 
##   functions makes sense because the smaller sub-routines could be used outside the scope of 
##   the main "load and process". As a rule of thumb: if you have around 20 or more lines in one 
##   function, you are probably doing too many things in the same block.
def load_and_process_directed_graph(filename):
    """Loads the network from a file, normalizes it, and calculates its page rank 
        (storing the value in the network itself)."""
    preProcessedGraph=load_directed_graph_from_file(filename)
    normalizedGraph=normalize_edge_weights(preProcessedGraph)
    pageRankedGraph=calculate_pagerank(normalizedGraph,TAU)
    return pageRankedGraph

### It is useful to define a function to this specific 'loading' task because you can use it outside
###    the "load and process" scope to do other analyses and tests.
def load_directed_graph_from_file(filename):
    "Imports a '.net' file and processes it as a directed network."
    directedGraph = nx.DiGraph(nx.read_pajek(filename))
    print(
      "Loaded a graph (%d nodes, %d edges)" % 
      (len(directedGraph), len(directedGraph.edges()))
    )
    return directedGraph

### Love this routine! So elegant.
def normalize_edge_weights(directedGraph):
    """For each node in the network: calculates the total of the weights' values,
        and then normalizes them to make their total equal to 1."""
    for node in directedGraph:
        edges = directedGraph.edges(node, data=True)
        total_weight = sum([data['weight'] for (_, _, data) in edges])
        for (_, _, data) in edges:
            data['weight'] = data['weight'] / total_weight
    return directedGraph
    
### I think it's useful to have 'tau' as an argument, even when it is defined as a global constant.
###   Even though the use of the global constant is completely justified in this case, having tau
###   as an argument for your function makes it much more flexible; and the typing is a small price
###   to pay for it.
def calculate_pagerank(normalizedGraph, tau):
    """Get the network's PageRank."""
    page_ranks = nx.pagerank(normalizedGraph, alpha=1-TAU)
    for (node, page_rank) in page_ranks.items():
        normalizedGraph.node[node][PAGE_RANK] = page_rank
    return normalizedGraph
    
## No changes. Just made the name a bit more specific.
def load_coordinates_from_file(filename):
    field_names = ['X','Y',"w"]
    coords = pd.read_csv(filename, header=None, names=field_names)
    coords = coords.loc[:,["X","Y"]]
    return coords

In [96]:
#####################################################################################################
# 'Unit-testing' against original functions #########################################################
#####################################################################################################
testProbability=.2
print("Log base 2: %r" % (log_base_two(testProbability)==log_base_two(testProbability)))
print("Partial entropy: %r" % (partial_entropy(testProbability)==entropy1(testProbability)))

graph=load_directed_graph_from_file("houses.net")
normalizedGraph=normalize_edge_weights(graph)
calculate_pagerank(normalizedGraph,TAU)

graphNew=load_and_process_directed_graph("houses.net")
partialEntropyNew=[partial_entropy(graphNew.node[node][PAGE_RANK]) for node in graph]
graphOld=load_and_process_graph("houses.net")
partialEntropyOld=[entropy1(graphOld.node[node][PAGE_RANK]) for node in graph]
print("Partial Entropy Network: %r" % (partialEntropyNew==partialEntropyOld))

loadedCoordinates=all(load_coordinates_from_file("coordinates.csv")==load_coordinates("coordinates.csv"))
print("Loaded coordinates: %r" % loadedCoordinates)

Log base 2: True
Partial entropy: True
Loaded a graph (20 nodes, 400 edges)
Loaded a graph (20 nodes, 400 edges)
Loaded a graph (20 nodes, 400 edges)
Partial Entropy Network: True
Loaded coordinates: True


In [24]:
# class Module:
#     """Stores the information about a single module"""
#     def __init__(self, module_id, nodes, graph):
#         self.module_id = module_id
#         self.nodes = frozenset(nodes)
#         self.graph = graph
#         self.prop_nodes = 1 - float(len(self.nodes)) / len(graph)
#         # Set the module_id for every node
# #         for node in nodes:
# #             graph.node[node][MODULE_ID] = module_id
#         # Compute the total PageRank
#         self.total_pr = sum([graph.node[node][PAGE_RANK] for node in nodes])
#         # Compute q_out, the exit probability of this module
#         # .. Left half: tau * (n - n_i) / n * sum{alpha in i}(p_alpha)
#         self.q_out = self.total_pr * TAU * self.prop_nodes
#         # .. Right half: (1-tau) * sum{alpha in i}(sum{beta not in i}
#         #                  p_alpha weight_alpha,beta)
#         # This is what's in [RAB2009 eq. 6]. But it's apparently wrong if
#         # node alpha has no out-edges, which is not in the paper.
#         # ..
#         # Implementing it with Seung-Hee's correction about dangling nodes
#         for node in self.nodes:
#             edges = graph.edges(node, data=True)
#             page_rank = graph.node[node][PAGE_RANK]
#             if len(edges) == 0:
#                 self.q_out += page_rank * self.prop_nodes * (1 - TAU)
#                 continue
#             for (_, dest, data) in edges:
#                 if dest not in self.nodes:
#                     self.q_out += page_rank * data['weight'] * (1 - TAU)
#         self.q_plus_p = self.q_out + self.total_pr

#     def get_codebook_length(self):
#         "Computes module codebook length according to [RAB2009, eq. 3]"
#         first = -entropy1(self.q_out / self.q_plus_p)
#         second = -sum( \
#                 [entropy1(self.graph.node[node][PAGE_RANK]/self.q_plus_p) \
#                     for node in self.nodes])
#         return (self.q_plus_p) * (first + second)

class GeoInfomap(Annealer):

    """Test annealer with a travelling salesman problem.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, module, graph, coordinates):
        self.graph = graph
        self.total_pr_entropy = sum([entropy1(graph.node[node][PAGE_RANK]) \
                for node in graph])
#         self.module = [Module(module_id, mod, graph) \
#                 for (module_id, mod) in enumerate(state)]
        d = 0
        for mod in module:
            for elem in range(len(mod)):
                mod[elem] = int(mod[elem])    
        for mod in module:
            m = coordinates.loc[mod,]
            d += np.mean(pairwise_distances(m, metric='euclidean'))
        self.d = d 

        super(GeoInfomap, self).__init__(state)  # important!
    def move(self):
        #converts list of node lists into a 1D array of community labels
        cluster_labels = []
        label = 0
        for cluster in self.state:
            for elem in cluster:
                cluster_labels.append(label)
            label += 1

    #flatten self.state
        flat_list = [item for sublist in self.state for item in sublist]
        #sort cluster labels list by node label (not community label)
        cluster_labels = [cluster_labels[flat_list.index(i)] for i in flat_list]
        a = random.randint(0, len(cluster_labels)-1)
        change_node = cluster_labels[a] 
        #if current label is 0, change to 1 to increase hamming distance by 1
        if change_node == 0:
            cluster_labels[a] = 1
        else:
            updown = random.randint(0, 1)
            if updown == 0:
                cluster_labels[a] -= 1
            cluster_labels[a] += 1

        #convert back to list of lists
        new_state = []
        labels = set(cluster_labels)
        for j in labels:
            cluster = []
            indices = [i for i, x in enumerate(cluster_labels) if x == j]
            cluster.extend(list(np.array(flat_list)[indices]))
            new_state.append(cluster)

        self.state = new_state

    def energy(self):
        "Compute the MDL of this clustering according to [RAB2009, eq. 4]"
        graph = self.graph
        
        total_qout = 0
        total_qout_entropy = 0
        total_both_entropy = 0
        for mod in self.state:
            nodes = frozenset(mod)
            prop_nodes = 1 - float(len(nodes)) / len(graph)
            total_pr = sum([graph.node[str(node)][PAGE_RANK] for node in nodes])
            q_out = total_pr * TAU * prop_nodes

            for node in mod:
                edges = graph.edges(str(node), data=True)
                page_rank = graph.node[str(node)][PAGE_RANK]
                if len(edges) == 0:
                    q_out += page_rank * prop_nodes * (1 - TAU)
                    continue
                for (_, dest, data) in edges:
                    if dest not in self.state:
                        q_out += page_rank * data['weight'] * (1 - TAU)
                q_plus_p = q_out + total_pr
        
            q_out = q_out
            total_qout += q_out
            total_qout_entropy += entropy1(q_out)
            total_both_entropy += entropy1(q_plus_p)
        term1 = entropy1(total_qout)
        term2 = -2 * total_qout_entropy
        term3 = -self.total_pr_entropy
        term4 = total_both_entropy
        term5 = self.d
        total =  term1 + term2 + term3 + term4 + 0.1*term5
        #print(total)
        return total

if __name__ == '__main__':
    #networkX Digraph
    graph = load_and_process_graph("houses.net")#(options.graph_filename)

    #coords is pandas dataframe of the coordinates
    coords = load_coordinates("coordinates.csv")
    coords.index = np.arange(0, len(coords))

    # single_nodes is the "trivial" module mapping
    # initial module, as a list of lists
    single_nodes = [[nodes] for nodes in graph]
    single_nodes[0] = ['0','1']
    single_nodes.remove(single_nodes[1]) 
    init_state = single_nodes

    gi = GeoInfomap(state = init_state, module = init_state, graph = graph, coordinates = coords)
    gi.steps = 10000
    # since our state is just a list, slice is the fastest way to copy
    gi.copy_strategy = "slice"
    state, e = gi.anneal()
        
    print()
   # print(state)
    print("%i mile route:" % e)
    for city in state:
        print("\t", city)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Loaded a graph (20 nodes, 400 edges)


     2.50000          8.95   100.00%    34.00%     0:00:16     0:00:00


8 mile route:
	 [0, 0, 1, 1]
	 [6]
	 [16, 14, 13]
	 [12, 8, 18, 3, 9, 17, 19, 4, 7, 2]
	 [15, 5]


In [44]:
from __future__ import print_function
import math
import random
from simanneal import Annealer


def distance(a, b):
    """Calculates distance between two latitude-longitude coordinates."""
    R = 3963  # radius of Earth (miles)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)) * R


class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """

    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important!

    def move(self):
        """Swaps two cities in the route."""
        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

    def energy(self):
        """Calculates the length of the route."""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e



if __name__ == '__main__':

    # latitude and longitude for the twenty largest U.S. cities
    cities = {
        'New York City': (40.72, 74.00),
        'Los Angeles': (34.05, 118.25),
        'Chicago': (41.88, 87.63),
        'Houston': (29.77, 95.38),
        'Phoenix': (33.45, 112.07),
        'Philadelphia': (39.95, 75.17),
        'San Antonio': (29.53, 98.47),
        'Dallas': (32.78, 96.80),
        'San Diego': (32.78, 117.15),
        'San Jose': (37.30, 121.87),
        'Detroit': (42.33, 83.05),
        'San Francisco': (37.78, 122.42),
        'Jacksonville': (30.32, 81.70),
        'Indianapolis': (39.78, 86.15),
        'Austin': (30.27, 97.77),
        'Columbus': (39.98, 82.98),
        'Fort Worth': (32.75, 97.33),
        'Charlotte': (35.23, 80.85),
        'Memphis': (35.12, 89.97),
        'Baltimore': (39.28, 76.62)
    }

    # initial state, a randomly-ordered itinerary
    init_state = list(cities.keys())
    random.shuffle(init_state)

    # create a distance matrix
    distance_matrix = {}
    for ka, va in cities.items():
        distance_matrix[ka] = {}
        for kb, vb in cities.items():
            if kb == ka:
                distance_matrix[ka][kb] = 0.0
            else:
                distance_matrix[ka][kb] = distance(va, vb)

    tsp = TravellingSalesmanProblem(init_state, distance_matrix)
    tsp.steps = 100000
    # since our state is just a list, slice is the fastest way to copy
    tsp.copy_strategy = "slice"
    state, e = tsp.anneal()

    while state[0] != 'New York City':
        state = state[1:] + state[:1]  # rotate NYC to start

    print()
    print("%i mile route:" % e)
    for city in state:
        print("\t", city)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000       6898.57     6.00%     0.10%     0:00:01     0:00:00


6882 mile route:
	 New York City
	 Columbus
	 Detroit
	 Chicago
	 Indianapolis
	 Memphis
	 Dallas
	 Fort Worth
	 Phoenix
	 San Francisco
	 San Jose
	 Los Angeles
	 San Diego
	 San Antonio
	 Austin
	 Houston
	 Jacksonville
	 Charlotte
	 Baltimore
	 Philadelphia


In [83]:
def move(self):
    """Swaps first element of two randomly chosen clusters."""

    #converts list of node lists into a 1D array of community labels
    cluster_labels = []
    label = 0
    for cluster in self:
        for elem in cluster:
            cluster_labels.append(label)
        label += 1

#flatten self.state
    flat_list = [item for sublist in self for item in sublist]
    #sort cluster labels list by node label (not community label)
    cluster_labels = [cluster_labels[flat_list.index(i)] for i in flat_list]
    print(cluster_labels)
    a = random.randint(0, len(cluster_labels)-1)
    print(a)
    change_node = cluster_labels[a] 
    #if current label is 0, change to 1 to increase hamming distance by 1
    if change_node == 0:
        cluster_labels[a] = 1
#         #if current label is maximal label, -1 from it
#         elif change_node == len(self.state):
#             cluster_labels[a] = len(self.state)-1
    #if current label is not 0 or maximal, increase or decrease by 1 with 0.5 prob
    else:
        updown = random.randint(0, 1)
        if updown == 0:
            cluster_labels[a] -= 1
        cluster_labels[a] += 1
    print(cluster_labels)
    #convert back to list of lists
    new_state = []
    labels = set(cluster_labels)
    for j in labels:
        cluster = []
        indices = [i for i, x in enumerate(cluster_labels) if x == j]
        cluster.extend(list(np.array(flat_list)[indices]))
        new_state.append(cluster)
    return (new_state)

In [84]:
move([[0, 2, 4], [1, 3], [5]])

[0, 0, 0, 1, 1, 2]
5
[0, 0, 0, 1, 1, 3]


[[0, 2, 4], [1, 3], [5]]